In [1]:
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt

from collections import Counter
from pprint import pprint

os.environ['http_proxy'] = "http://proxy-ws.cbank.kz:8080"
os.environ['https_proxy'] = "http://proxy-ws.cbank.kz:8080"

ModuleNotFoundError: No module named 'torch'

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))

cuda device is available


In [14]:
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
    
with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()
    
text = "".join([x.replace('\t\t', '').lower() for x in text])

--2024-11-27 05:19:53--  https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
Resolving proxy-ws.cbank.kz (proxy-ws.cbank.kz)... 10.1.246.2
Connecting to proxy-ws.cbank.kz (proxy-ws.cbank.kz)|10.1.246.2|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: ‘onegin.txt.2’

onegin.txt.2        100%[===================>] 256.37K   827KB/s    in 0.3s    

2024-11-27 05:19:53 (827 KB/s) - ‘onegin.txt.2’ saved [262521/262521]



In [15]:
text[:100]

'\ni\n\n«мой дядя самых честных правил,\nкогда не в шутку занемог,\nон уважать себя заставил\nи лучше выдум'

### Создадим буквенно-символьные токены и дадим им индексы

In [16]:
tokens = sorted(set(text.lower())) + ['<sos>']

token_to_index = { x : index for index, x in enumerate(tokens) }
index_to_token = { index : x for index, x in enumerate(tokens) }

assert len(tokens) == 84, "Check the tokenization process"
assert len(tokens) == len(token_to_index)

text_encoded = [token_to_index[token] for token in text]

In [17]:
print(text_encoded[:10])

[0, 26, 0, 0, 43, 57, 59, 54, 1, 49]


#### Рандомные чанки

In [18]:
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_index['<sos>']

In [43]:
def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data)).astype(int)

In [ ]:
chunk = next(generate_chunk())  

In [45]:
chunk.shape

(256, 101)

In [46]:
chunk

array([[83, 53, 58, ..., 66,  7,  0],
       [83, 47,  1, ...,  5,  1, 49],
       [83, 45, 47, ..., 61, 59, 49],
       ...,
       [83, 61,  1, ..., 47, 56, 45],
       [83, 49, 53, ..., 58, 59, 69],
       [83, 45,  6, ..., 45, 56,  1]])

In [48]:
batch = torch.LongTensor(chunk)

batch

tensor([[83, 53, 58,  ..., 66,  7,  0],
        [83, 47,  1,  ...,  5,  1, 49],
        [83, 45, 47,  ..., 61, 59, 49],
        ...,
        [83, 61,  1,  ..., 47, 56, 45],
        [83, 49, 53,  ..., 58, 59, 69],
        [83, 45,  6,  ..., 45, 56,  1]])

In [49]:
embed = torch.nn.Embedding(seq_length + 1, 256)

In [23]:
# x_input = (1, input_size)
# W_input_hidden = (input_size, hidden_size)

# current_state = x_input * W_input_hidden = (1, hidden_size)

# last_state = (1, hidden_size)
# W_hidden_hidden = (hidden_size, hidden_size)

# from_last_state = last_state * W_hidden_hidden
# from_last_state = (1, hidden_size)

# go_to_next_state = current_state + from_last_state

# h_init = (1, hidden_size) from normal(0, 1)



In [84]:
class RNNcell(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.embed = torch.nn.Embedding(input_size, 112)
        self.W = torch.nn.Linear(hidden_size + input_size, hidden_size)
        # self.W_classify = torch.nn.Linear(hidden_size, num_tokens)
        
    def forward(self, prev_hidden, input_X):
        # input_X = (batch_size, input_size)
        print("prev_hidden: ", prev_hidden.shape)
        print("input_X: ", input_X.shape)

        embedding = embed(input_X)

        print("embedding size: ", embedding.shape)
                
        return embedding

In [85]:
HIDDEN_SIZE = 50

In [86]:
rnn_cell = RNNcell(seq_length + 1, HIDDEN_SIZE)

In [87]:
init_state = torch.zeros(1, HIDDEN_SIZE)

In [89]:
res = rnn_cell.forward(init_state, batch[0])

prev_hidden:  torch.Size([1, 50])
input_X:  torch.Size([101])
embedding size:  torch.Size([101, 256])


In [65]:
res

1

In [81]:
chunk

array([[83, 53, 58, ..., 66,  7,  0],
       [83, 47,  1, ...,  5,  1, 49],
       [83, 45, 47, ..., 61, 59, 49],
       ...,
       [83, 61,  1, ..., 47, 56, 45],
       [83, 49, 53, ..., 58, 59, 69],
       [83, 45,  6, ..., 45, 56,  1]])

In [ ]:
# input_size = (batch_size, input_size)
# Embedding_size = (input_size, embed_size)
